# Live Face-Mask Detection - Using YOLO pretrained model
# conda activate yolo

In [1]:
import cv2
from ultralytics import YOLO
import pygame
import time

# Load custom face mask detection YOLO model
# model = YOLO("/media/akashs/FA22E72622E6E69B/DATA1_disk/MyGithub/Mask_Detection/archive/runs/detect/train2/weights/best.pt")  # Change to your model path
model = YOLO("./best.pt")  # Change to your model path
names = model.names  # Should include 'mask' and 'no_mask'


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
names

{0: 'with_mask', 1: 'without_mask', 2: 'mask_weared_incorrect'}

### Detection

In [3]:

# Access default live-cam (0)
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("mask_video2.mp4")

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_filename = f"recordings/recorded_{time.strftime('%Y%m%d_%H%M%S')}.avi"
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_filename, fourcc, 20.0, (frame_width, frame_height))


img_counter = 0
print("Live cam started. Press 's' to save image, 'q' to quit.")

# Optional sound setup
pygame.mixer.init()
alert_sound = pygame.mixer.Sound("chime_alert.mp3")

alert_triggered = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Live cam frame not received")
        break

    # -----------------------------
    # Draw Line, Rectangle, Circle, Text
    # -----------------------------
    height, width = frame.shape[:2]

    results = model(frame)

    # 🔹 Extract results
    boxes = results[0].boxes
    names = model.names  # COCO class names

    # Uses OpenCV's haarcascade
    # https://github.com/opencv/opencv/tree/master/data/haarcascades
    
    detected_classes = [int(box.cls[0].item()) for box in boxes]
    labels = [model.names[cls_id] for cls_id in detected_classes]

    # Draw detection boxes and play alert for "without_mask"
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        label_text = f"{label}: {conf:.2f}"

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 0, 0), 2)
        print(f"🟢 Detected: {label_text} at ({x1}, {y1}, {x2}, {y2})")

        # Play sound for each "without_mask" detected
        if "without_mask" in label.lower():
            alert_sound.play()

        
    # -----------------------------
    # Save frame to video file
    # -----------------------------
    out.write(frame)

    # -----------------------------
    # Show live-cam feed
    # -----------------------------
    cv2.imshow('Live-cam Feed with Drawing', frame)

    # -----------------------------
    # Keypress handling
    # -----------------------------
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        print("Exiting live-cam...")
        break
    elif key & 0xFF == ord('s'):
        # Save snapshot image
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        img_filename = f"snapshot_{timestamp}.jpg"
        cv2.imwrite(img_filename, frame)
        print(f"Snapshot saved: {img_filename}")
        img_counter += 1

# -----------------------------
# Release resources
# -----------------------------
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Recorded video saved as: {output_video_filename}")


Live cam started. Press 's' to save image, 'q' to quit.

0: 480x640 (no detections), 34.7ms
Speed: 2.6ms preprocess, 34.7ms inference, 16.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.9ms
Speed: 1.9ms preprocess, 4.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.6ms
Speed: 1.6ms preprocess, 6.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 with_mask, 6.8ms
Speed: 1.7ms preprocess, 6.8ms inference, 86.4ms postprocess per image at shape (1, 3, 480, 640)
🟢 Detected: with_mask: 0.42 at (153, 198, 358, 406)

0: 480x640 (no detections), 5.9ms
Speed: 2.1ms preprocess, 5.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 with_mask, 5.6ms
Speed: 1.3ms preprocess, 5.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
🟢 Detected: with_mask: 0.77 at (211, 193, 386, 400)

0: 480x640 1 with_mask, 1 without_mask, 6.0ms
Speed: 1.5m